# Step 0: Antenna design
## Recommendations
### EMC
You can add a ground trace on another layer that covers the traces in order for common mode currents to capacitevely couple to the ground plane.  Do not close the loop of that ground trace, because that would cause eddy currents that short the antenna.

### Antenna size
* Larger antennas have larger magnetic field strength for the same input power
* Maximum coupling is reached when coil radius equals reading distance.
* Maximum achievable coupling decreases as antenna size increases.
* Larger antennas can generate the minimum field strength (1.5A/m) over longer reading distances, but they require more current to do it.
* Minimum antenna size: 500mm² (25mm x 20mm)
* Recommended antenna size : 40mm x 40mm
* Maximum antenna size: 5000mm² (71mm x 71mm)

### Number of turns 
<img src="./inductance_versus_coil_radius.png" width="600px">
Remark that the x-axis is **radius** and not diameter. 

<img src="./max_coupling_vs_antenna_size.png" width="600px">
Remark that the x-axis is **diameter** and not radius.

Maximum coupling is reaced for a coil with a diameter d=4cm (on X-axis), and a reading distance X=2cm.  It's not given what the size of the PICC is, probably something about the same size as the PCD.

* You can not increase operating distance by changing the number of turns.
* Typical number of turns: 1-4 turns (PN7150: 2-8 turns)
* effect on matching circuit : for load modulation (the NFC-card does), more turns are better

### Antenna inductance
* Antenna inductance: 700nH ~ 1.6μH
* Target antenna inductance (for matching) : 1µH → 3µH, best 1µH, L ≈ 300nH up to L≈ 4μH still can be matched properly

### Antenna impedance
* Antenna matching network R_match should be around 20 - 25 Ohm.  That's a 250mA current for a 5V supply.

### Antenna Q-factor
* Q-factor should be around 10-20 for ISO14443 communication.

### Various
* Matching circuit should be close to the antenna, not close to the reader.
* https://www.nxp.com/docs/en/application-note/AN11755.pdf : ferrite shielding
* Minimum track spacing : 0.2mm
* Minimum track width : 0.2mm

## Antenna design for rectangular coil

In [1]:
import math

d_in = 45e-3
d_out = 50e-3
N = 3 # number of turns

K1 = 2.34 # constant for a square coil
K2 = 2.75 # constant for a square coil
µ_0 = 4e-7 * math.pi # permeability of free space in H/m
d = (d_in + d_out)/2
ρ = (d_out - d_in)/(d_out + d_in) # filling ratio
L_ant = K1 * µ_0 * N**2 * d / (1 + K2*ρ)
print(f"L_ant = {L_ant*1e6:.2f} µH")

L_ant = 1.10 µH


# Step 1: Antenna model measurement
## VNA setup
* start frequency: 1 MHz
* stop frequency: > self resonant frequency (SRF) of the antenna (100 MHz?)

SRF on the Smith chart is the frequency where the impedance curve crosses the horizontal axis.

## Measurement of parameters

In [2]:
import math
L_a = 1.27e-6 # H, Equivalent inductance at f = 1MHz
R_s = 2.18 # Ohm, Equivalent resistance at f = 1MHz

f_ra = 49.8e6 # Hz, Self-resonance frequency of the antenna
R_p = 2.2e3 # Ohm, Equivalent resistance at the self-resonance frequency

f_op = 13.56e6 # Hz, Operating frequency
ω_op = 2*math.pi*f_op # rad/s, Operating angular frequency

<img src="./series_equivalent_calculation.png" width="400"/>

In [3]:
# Calculation of equivalent circuit of the antenna
C_a = 1 / ((2 *  math.pi * f_ra)**2 * L_a) # F, antenna capacitance
print(f"C_a = {C_a*1e12:.2f} pF")
R_p_op = R_p / math.sqrt(f_op / f_ra) # Ohm, Equivalent resistance at the operating frequency
R_a = R_s + (ω_op * L_a)**2 / R_p_op # Ohm, Equivalent resistance at the operating frequency
print(f"R_a = {R_a:.2f} Ohm")

# Evaluation of measured parameters
if(L_a < 0.3e-6 or L_a > 3e-6):
    print("L_a is out of range.")
if(C_a < 3e-12 or C_a > 30e-12):
    print("C_a is out of range.")
if(f_ra < 25e6):
    print("f_ra is out of range.")
# There's no check on R_a because it's the Q-factor that is important.      


C_a = 8.04 pF
R_a = 4.96 Ohm


## Quality factor
A correct range for the Q-factor is 20-35.
Series resistors R_q can be added to lower the Q-factor when needed.

It's a pain to tune a high Q-factor antenna.  Insert series resistors to lower the Q-factor.  Remeasure the basic parameters until the Q-factor is around 20.  Then proceed with the matching circuit design.

In [4]:
# Calculation of the antenna Q factor
Qa = ω_op * L_a / R_a
if(Qa < 20 or Qa > 35):
    print("The antenna Q factor is not in the range of 20 to 35.")
    R_q = max(0.5 * (ω_op * L_a / 35 - R_a), 0) # Ohm, Equivalent resistances (2x) to be added to the antenna to make the Q factor 35
    print(f"R_q = {R_q:.2f} Ohm")
else:
    R_q = 0
print(f"Qa = {Qa:.2f}")

Qa = 21.83


## Parallel equivalent circuit
<img src="./parallel_equivalent_circuit.png" width="400"/>

In [5]:
L_pa = L_a
C_pa = C_a
R_pa = (ω_op * L_a)**2 / (R_a + 2 * R_q)
print(f"L_pa = {L_pa*1e6:.2f} µH")
print(f"C_pa = {C_pa*1e12:.2f} pF")
print(f"R_pa = {R_pa:.2f} Ohm")

L_pa = 1.27e-6 # H, Equivalent inductance at f = 1MHz
C_pa = 8.0e-12 # F, Equivalent capacitance at f = 1MHz
R_pa = 3.06e3 # Ohm, Equivalent resistance at f = 1MHz

L_pa = 1.27 µH
C_pa = 8.04 pF
R_pa = 2361.92 Ohm


# Step 2: EMC filter design (L0 and C0 definition)
* L_0 should be in the range of 160nH to 560nH (NXP).
* L_0 range should be around 390nH to 680nH (Holtek).
* f_r0 filter resonance frequency should be in the range of 15.5MHz to 17MHz (AN11740 has another opinion on this).

In [6]:
L_0 = 470e-9 # H, Inductance of the matching network
C_0 = 150e-12 # F, Capacitance of the matching network

f_r0 = 1 /  (2 * math.pi * math.sqrt(C_0 * L_0)) # Hz, Resonance frequency of the matching network
print(f"The resonance frequency is {f_r0/1e6:.2f} MHz")

The resonance frequency is 18.96 MHz


## Impedance transformation
<img src="./impedance_transformation.png" width="600"/>

The EMC filter and the matching network must transform the antenna impedance Z_match(f) to the required TX matching resistance R_match at the operating frequency of f = 13.56 MHz.

## Calculation of impedance matching network
<img src="./definition_of_transformation_impedance_Ztr.png" width="600"/>

By cutting the circuitry after the EMC filter and by using the precondition R match/2, the remaining components C_1 and C_2 can be calculated.
Optimal power transfer is achieved through conjugate matching.  The impedance looking into the EMC-filter Z_tr should be the complex conjugate of the impedance looking into the impedance matching network Z_tr*.

In [7]:
R_match = 20 # Ohm, Desired load resistance on TX1-TX2
a = (1 - ω_op**2 * L_0 * C_0) # helper variable
b = a**2 + (ω_op*R_match*C_0/2)**2 # helper variable
R_tr = R_match / b # Real part of Z_tr
X_tr = 2 * ω_op * (L_0 * a - (R_match**2)*C_0/4) / b # Imaginary part of Z_tr

# Step 3: Reader mode matching (C1 and C2 definition)

In [8]:
# C1 only depends on R_pa of the antenna
C_1 = 1 / (ω_op * (math.sqrt(R_tr*R_pa/4)+X_tr/2))
# C2 depends on L_pa, C_pa and R_pa
C_2 = 2 / (ω_op**2 * L_pa) - 1 / (ω_op * math.sqrt(R_tr*R_pa/4)) - 2 * C_pa

print(f"C_1 = {C_1*1e12:.2f} pF")
print(f"C_2 = {C_2*1e12:.2f} pF") 

C_1 = 37.05 pF
C_2 = 153.05 pF


# Step 4 : tuning of the matching network
Work your way from the antenna to the reader.  

1. Remove all matching components and the LC filter (L0 and C0).
2. Measure the antenna impedance Z_antenna at 13.56 MHz and calculate L_pa, C_pa and R_pa.
3. Tune your simulation model (only R_pa) to match the measured impedance at 13.56 MHz.
4. With the correct antenna model, you can calculate C1 and C2. And maybe adjust them to get 20 Ohm differential impedance (or 10 Ohm single-ended impedance) and a 0° phase shift at 13.56 MHz.
4. Add C2 in the model of the antenna.  There will be a resonance peak, e.g. at 15.5 MHz.
5. Add C2 in the PCB and adjust the value until the resonance peak is at the same frequency as in the simulation.
6. Add C1 in the model of the antenna and measure the impedance at 13.56 MHz.
7. Add C1 in the PCB and adjust the value until you get the same impedance as in the simulation.
8. Add L0 and C0 in the model of the antenna and measure the impedance at 13.56 MHz.
9. If the phase of the impedance is not 0°, adjust C1.  Increasing C1 will increase the phase, decreasing C1 will decrease the phase.
10. If the magnitude of the impedance is not 20 Ohm, adjust C2.  Increasing C2 will increase the impedance, decreasing C2 will decrease the impedance.

# References
1. [NXP AN11755](https://www.nxp.com/docs/en/application-note/AN11755.pdf)
2. Holtek AN0650EN
3. [NFC-Antenna design](https://community.nxp.com/t5/NFC-Knowledge-Base/NFC-Antenna-Design/ta-p/1112414)
4. [AN11740](https://www.nxp.com/docs/en/application-note/AN11740.pdf)
5. [DPC antenna tuning](https://www.nxp.com/docs/en/application-note/AN11741.pdf)
6. [using NanoVNA for NFC antenna design](https://www.nxp.com/docs/en/application-note/AN12810.pdf)
7. [antenna with components and ground plane on the inside](https://www.ti.com/lit/an/sloa197/sloa197.pdf)
8. [PICC Class 1 antenna](https://www.comprion.com/shop/antennas/emvco-3-0-test-picc-1.html)

Example antenna from Eccel: 80x80mm, 2 turns

In [9]:
# Holtek antenna tuning
f_op = 13.56e6 # Hz, Operating frequency
C2 = 30e-12 # F, fixed capacitor
L1= 470e-9 # H, fixed inductor
Lant = 1.877e-6 # H, antenna inductance
C1 = 1/((2*math.pi*f_op)**2 * L1) - C2
print(f"C1 = {C1*1e12:.2f} pF")
C3 = 2 /((2*math.pi*f_op)**2 * Lant) - C2
print(f"C3 = {C3*1e12:.2f} pF")

C1 = 263.10 pF
C3 = 116.79 pF
